## Imports

In [1]:
#from rk4step import rk4step
import numpy as np
import casadi as ca
import random
from scipy.stats import rv_continuous, rv_discrete

## Constants

In [2]:
P_e = 12
P_i = 10
C = 50
M = 5
dim_O = 20
dim_V = 20

U = [-4, -3, -2, -1,0,1,2, 3,4]
O_range = list(range(dim_O))
V_range = list(range(dim_V))
V_change_range = 5
V_change = np.array([-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5])

## Help functions

In [3]:
class UniformDiscretDistr(rv_discrete):
    def _pmf(self, k):
        return 1./(self.b + 1 - self.a)

In [4]:
uniform = UniformDiscretDistr(a=-V_change_range, b=V_change_range, name="uniform")

In [5]:
def L_e(o0, o1, v):
    if(o0 >= v and o1 >= v):
        return 0
    elif(o0 < v and o1 < v):
        return 0.5*(v - o0 + v - o1) * P_e
    else:
        t = (v - o0)/(o1-o0)
        if(o0 < v and o1 >= v):
            return 0.5 * (o1 - v) * (1-t) * P_e
        elif(o0 >= v and o1 < v):
            return 0.5 * (o0 - v) * t * P_e 

def L_i(o0, o1):
    return 0.5 * (o0 + o1) * P_i
"""

def V(v):
    return v * C

def O_i(k, O_i_prev, u):
    return O_i_prev + u * k

def L_i(k, O_i_prev, u):
    return P_i * O_i(k, O_i_prev, u)

def O_e(k, O_i_prev, u, v):
    return np.max([V(v) - O_i(k, O_i_prev, u), 0])

def L_e(k, O_i_prev, u, v):
    return P_e * O_e(k, O_i_prev, u, v)

# integrator nicht mehr wrong, aber wahrscheinlich ineffizient
def L(x0, u, v):
    x = ca.SX.sym('x')
    k = ca.SX.sym('k')
    ode = {'x':x, 't':k,'ode':P_e * ca.fmax(v * C - x0[0] + u * k, 0) + P_i * (x0[0] + u * k)}
    F = ca.integrator('F', 'idas', ode,{'t0':0,'tf':2})
    r = F(x0=0)
    return(r['xf'])
""";

In [18]:
class GridOptimizer:
    def __init__(self, dim, L_i, L_e):
        #self.current_state = None
        self.dim = dim
        self.L_i = L_i
        self.L_e = L_e
        self.max_cost_to_go_m = np.zeros(self.dim, dtype=float)
        self.max_iter_depth = 0
        self.max_opt_dec_m = None
        
    def f(self, x0, u, v):
        """
        Calculates next state based on decision and new consum.
        """
        o1 = x0[0] + u
        v1 = min(self.dim[1]-1, max(x0[1] + v, 0))
        if 0 > o1 or o1 >= self.dim[1]:
            return None
        return (o1, v1)

    def L(self, x0, x1):
        return self.L_i(x0[0], x1[0]) + L_e(x0[0], x1[0], x1[1])

    def calculate_cost_to_go_matrix_sequence(self, depth):
        if self.max_iter_depth < depth:
            #M = np.zeros((2, self.dim[0],self.dim[1]), dtype=float)
            choice = np.zeros(self.dim, dtype=int)
            M = [np.zeros(self.dim, dtype=float), np.zeros(self.dim, dtype=float)]
            M[0] = self.max_cost_to_go_m
            for i in range(depth - self.max_iter_depth+1):
                if(i != 0):
                    M[i%2], choice = self.calculate_cost_to_go_matrix(M[(i-1)%2])
                else:
                    if self.max_iter_depth > 0:
                        continue
                    M[i] = self.calculate_cost_to_go_matrix_final_step()
            self.max_cost_to_go_m =  M[(depth - self.max_iter_depth)%2]
            self.max_iter_depth = depth
            self.max_opt_dec_m = choice

    # calculate matrix with includes indexes for actions U that are optimal for given x0 in matrix
    def calculate_optimal_step_matrix(self, depth):
        if self.max_iter_depth < depth:
            self.calculate_cost_to_go_matrix_sequence(depth=depth)
        '''optimal_steps = np.zeros(self.dim, dtype=int)
        for o_index in range(self.dim[0]):
            for v_index in range(self.dim[1]):
                step_cost_to_go_array = np.zeros(len(U), dtype=float)
                for u_index in range(len(U)):
                    x0 = (O_range[o_index], V_range[v_index])
                    step_cost_to_go_array[u_index] = self.calculate_path_cost(x0, U[u_index], self.max_cost_to_go_m)
                min_index_u = 0
                for i in range(1, len(step_cost_to_go_array)):
                    if (step_cost_to_go_array[i] < step_cost_to_go_array[min_index_u]):
                        min_index_u = i
                optimal_steps[o_index, v_index] = U[min_index_u]
        self.max_opt_dec_m =  optimal_steps'''

    # TODO change end score
    def calculate_cost_to_go_matrix_final_step(self):
        M = np.ndarray(self.dim, dtype=float)
        for o_index in range(dim_O):
            for v_index in range(dim_V):
                M[o_index,v_index] = 0
                # end costs are zeros at the moment, possible change
                # e.g. M[o_index,v_index] = np.abs(o_index - v_index) * (2 if o_index - v_index > 0 else 1)
        return M

    # TODO optimieren mit list comprehension?
    def calculate_cost_to_go_matrix(self, cost_matrix):
        M = np.ndarray(self.dim, dtype=float)
        choice = np.ndarray(self.dim, dtype=int)
        for o_index in range(self.dim[0]):
            for v_index in range(self.dim[1]):
                x0 = (O_range[o_index], V_range[v_index])
                M[o_index,v_index], choice[o_index,v_index] = self.cost_to_go(x0, cost_matrix)
        return M, choice
        # [[cost_to_go((o,v), U, cost_matrix) for o in range(dim_O)] for v in range(dim_V)]

    def cost_to_go(self, x0, cost_matrix):
        step_cost_to_go_array = np.zeros(len(U), dtype=float)
        for u_index in range(0,len(U)):
            step_cost_to_go_array[u_index] = self.calculate_path_cost(x0, U[u_index], cost_matrix)
        # step_cost = [self.calculate_path_cost(x0, U[u_index], cost_matrix) for u in U]
        # min_index = np.argmin(step_cost)
        # return step_cost[min_index], U[min_index]
        return np.min(step_cost_to_go_array), U[np.argmin(step_cost_to_go_array)]

    #TODO ERWARTUNGSWERT
    def calculate_path_cost(self, x0, u, cost_matrix):
        # Calculate costs of one decision as expected value with random variable "next possible state".
        if 0 <= x0[0] + u < self.dim[0]:
            # calculate expected value
            summ = 0
            num_v_in_range = 0
            #rv_f = lambda k: self.L(x0, self.f(x0, u, k)) + cost_matrix[self.f(x0, u, k)]
            def rv_f(k):
                if isinstance(k, np.ndarray):
                    ls = []
                    for i in k:
                        ls.append(rv_f(i))
                    return np.array(ls)
                x1 = self.f(x0, u, k)
                return self.L(x0, x1) + cost_matrix[x1]
            return uniform.expect(rv_f)
            # currently uniform distribution over all V capped at [0, dim_V]
            # so it is no uniform distribution!!
        return np.inf

In [24]:
grid_opt = GridOptimizer(dim=(dim_O, dim_V), L_i=L_i, L_e=L_e)

In [25]:
cur_mat = []
for i in range(5):
    print(i)
    #grid_opt.calculate_cost_to_go_matrix_sequence(depth = i)
    grid_opt.calculate_optimal_step_matrix(depth = i * 2 + 1)
    #print(grid_opt.max_cost_to_go_m)
    #cur_mat.append(grid_opt.max_opt_dec_m)
    print(grid_opt.max_opt_dec_m)

0
[[ 0  0  0  0  2  3  4  4  4  4  4  4  4  4  4  4  4  4  4  4]
 [-1 -1 -1 -1 -1  2  3  4  4  4  4  4  4  4  4  4  4  4  4  4]
 [-2 -2 -2 -2 -2 -2  2  3  4  4  4  4  4  4  4  4  4  4  4  4]
 [-3 -3 -3 -3 -3 -3 -3  2  3  4  4  4  4  4  4  4  4  4  4  4]
 [-4 -4 -4 -4 -4 -4 -4 -4  2  3  4  4  4  4  4  4  4  4  4  4]
 [-4 -4 -4 -4 -4 -4 -4 -4 -4  2  4  4  4  4  4  4  4  4  4  4]
 [-4 -4 -4 -4 -4 -4 -4 -4 -4 -4  2  3  4  4  4  4  4  4  4  4]
 [-4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4  2  3  4  4  4  4  4  4  4]
 [-4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4  2  3  4  4  4  4  4  4]
 [-4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4  2  3  4  4  4  4  4]
 [-4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4  2  3  4  4  4  4]
 [-4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4  2  3  4  4  4]
 [-4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4  2  3  4  4]
 [-4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4  2  3  4]
 [-4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4  2  4]
 [-4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 

In [26]:
for i in range(len(cur_mat)-1):
    if np.any(cur_mat[i]-cur_mat[i+1]):
        print("change")
    else:
        print("no_change")

In [11]:
def look_up_table(dim = (3, 5, 10)):
    n = len(dim)
    ls = []
    for i in range(n):
        transposition = list(range(n))
        transposition[i], transposition[n-1] = n-1, i
        x = np.transpose(np.transpose(np.zeros(dim, dtype=int), axes=transposition) + np.arange(dim[i]), axes=transposition)
        ls.append(x)
    return np.stack(ls, axis=n)


array([0, 6, 3, 2, 8])